<a href="https://colab.research.google.com/github/Maxxx-VS/The-Founder/blob/master/25_4_Lenta_ru_stop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

В домашней работе необходимо с помощью AutoKeras или KerasTuner найти оптимальную модель для решения одной из следующей задач:

1. На 3 балла. Обучите модель с точностью не менее 90% предсказывать сарказм в новостных заголовках. Составьте 5 произвольных заголовков, которых нет в датасете и проверьте на них обученную модель, сделайте выводы. Ссылка на [датасет](https://storage.yandexcloud.net/academy.ai/Sarcasm_Headlines_Dataset_v2.json.zip).
2. На 4 балла. Используйте [русский корпус новостей от Lenta.ru](https://www.kaggle.com/datasets/yutkin/corpus-of-russian-news-articles-from-lenta/data) подберите и обучите модель классифицировать новости по заголовкам на классы (поле topic в датасете). Используйте 9 самых часто встречаемых топиков и 10-й для остальных, не вошедших в 9 классов. Оцените модель с помощью отчета о классификации, сделайте выводы.  
3. На 5 баллов. Найдите публичный датасет по обращениям граждан в администрацию, техническую поддержку или за консультацией. Обучите модель классифицировать обращения по тематикам. Сформируйте отчет о классификации и матрицу ошибок.

In [ ]:
!pip install autokeras==1.1.0 tensorflow==2.15.1 keras-nlp==0.5.1

INFO: pip is looking at multiple versions of tensorflow-text to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of tf-keras to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.6/148.6 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.1/527.1 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 52.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 59.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 69.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.0/442.0 kB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.7 MB/s eta 

In [ ]:
import numpy as np
import pandas as pd
import re
import tensorflow as tf
import autokeras as ak
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import kagglehub

In [ ]:
!wget https://storage.yandexcloud.net/for-telegram-bot2/Russian_news_corps_lenta_ru.zip

--2024-11-01 06:03:19--  https://storage.yandexcloud.net/for-telegram-bot2/Russian_news_corps_lenta_ru.zip
Resolving storage.yandexcloud.net (storage.yandexcloud.net)... 213.180.193.243, 2a02:6b8::1d9
Connecting to storage.yandexcloud.net (storage.yandexcloud.net)|213.180.193.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 611887272 (584M) [application/x-zip-compressed]
Saving to: ‘Russian_news_corps_lenta_ru.zip’

Russian_news_corps_ 100%[===================>] 583.54M  21.5MB/s    in 28s     

2024-11-01 06:03:48 (20.9 MB/s) - ‘Russian_news_corps_lenta_ru.zip’ saved [611887272/611887272]



In [ ]:
!unzip -qo "Russian_news_corps_lenta_ru.zip" -d ./dataset

In [ ]:
data_path = "/content/dataset/lenta-ru-news.csv"
df = pd.read_csv(data_path)

<ipython-input-5-33e41decfc57>:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_path)


In [ ]:
df = df[['topic', 'title']]

In [ ]:
df.drop_duplicates(subset='title', inplace=True)

In [ ]:
topics = df['topic'].value_counts().index[:9]
df.loc[~df['topic'].isin(topics), 'topic'] = 'Other'

In [ ]:
topic_num = {topic: idx for idx, topic in enumerate(df['topic'].unique())}
df['topic_num'] = df['topic'].map(topic_num)

In [ ]:
X_train, X_tmp, y_train, y_tmp = train_test_split(np.array(df['title']), np.array(df['topic_num']),test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_tmp, y_tmp, test_size=0.5)

In [ ]:
clf = ak.TextClassifier(overwrite=True, max_trials=1, objective='val_accuracy')
result_training = clf.fit(X_train, y_train, epochs=2, validation_data=(X_val, y_val))

Trial 1 Complete [02h 32m 26s]
val_accuracy: 0.5867540836334229

Best val_accuracy So Far: 0.5867540836334229
Total elapsed time: 02h 32m 26s
Epoch 1/2
19946/19946 [==============================] - 4541s 228ms/step - loss: 1.3440 - accuracy: 0.5481 - val_loss: 1.2129 - val_accuracy: 0.5826
Epoch 2/2
 1231/19946 [>.............................] - ETA: 1:08:16 - loss: 1.2642 - accuracy: 0.5763

KeyboardInterrupt: 

In [ ]:
clf.export_model().summary()

In [ ]:
predicted_y = clf.predict(X_test)
print(clf.evaluate(X_test, y_test))

In [ ]:
def plot_graphs(history, key):
    sns.set_style('darkgrid')
    palette = sns.color_palette('Set1')
    plt.figure(figsize=(6,3))
    plt.plot(history.history[key],   # данные для построения графика
             marker='o',             # использовать маркер для точек на графике
             color=palette[6],       # использовать цвет третий в палитре (нумерация с 0)
             linewidth=2.5,          # ширина линии
             label=key)              # подпись легенды

    if 'val_'+key in history.history:
        plt.plot(history.history['val_'+key], # данные для построения графика
                 marker='o',                  # использовать маркер для точек на графике
                 color=palette[2],            # использовать цвет пятый в палитре
                 linewidth=2.5,               # ширина линии
                 label='val_'+key)            # подпись легенды

    plt.xlabel("Эпоха", fontsize=6)
    plt.ylabel(key, fontsize=6)
    plt.title('Обучающая и проверочная выборки ('+key.title()+')', fontsize=8)
    plt.legend(fontsize=6)
    plt.grid(True)
    plt.show()

In [ ]:
plot_graphs(result_training, "accuracy")
plot_graphs(result_training, "loss")

In [ ]:
print(classification_report(y_test, predicted_y))

In [ ]:
def predict_sarcasm(text, model):
    result = model.predict(np.array([text]))
    if result[0] == 0:
        print("Предложение: '{}' не содержит сарказма.".format(text))
    elif result[0] == 1:
        print("Предложение : '{}' содержит сарказм.".format(text))

sarcasm = "Oh, wonderful! My computer crashes right before I save my work."
without_sarcasm = "I love spending time with my family during the holidays."

predict_sarcasm(sarcasm, clf)
predict_sarcasm(without_sarcasm, clf)

In [ ]:
input_node = ak.TextInput()
output_node = ak.TextBlock()(input_node)
output_node = ak.ClassificationHead()(output_node)
clf_2 = ak.AutoModel(inputs=input_node, outputs=output_node, overwrite=True,  max_trials=2, objective='val_accuracy')
# Обучаем 2 пободранных модели на 3 эпохах
res_2 = clf_2.fit(X_train, y_train, epochs=3)

In [ ]:
clf_2.export_model().summary()

In [ ]:
predict_sarcasm(sarcasm, clf_2)
predict_sarcasm(without_sarcasm, clf_2)

In [ ]:
!pip install keras-tuner

In [ ]:
import keras
from keras import layers
from kerastuner.tuners import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters
from tensorflow.keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences

In [ ]:
vocab_size = 10000
sentence_len = 25

# Функция предобработки данных
def data_preparation(data):
    corpus=[]

    for text in data:
        text=re.sub("[^a-zA-Z]", " ", text)
        text=text.lower()
        text=text.split()
        text=" ".join(text)
        corpus.append(text)

    # Векторизуем текст
    oneHot_doc=[one_hot(input_text=words, n=vocab_size)
               for words in corpus]
    embedded_doc=pad_sequences(sequences=oneHot_doc, maxlen=sentence_len, padding="post")
    return embedded_doc

In [ ]:
# Запускаем предобработку данных
train_data = data_preparation(X_train)
test_data = data_preparation(X_test)

In [ ]:
# Вывод первого заголовка обучающей выборки в исходном виде
X_train[0]

In [ ]:
# Вывод первого заголовка обучающей выборки в векторизованном виде
train_data[0]

In [ ]:
# Определение функции для построения модели
def build_model(hp):
    model = keras.Sequential()
    # Добавление слоев эмбеддинга
    model.add(layers.Embedding(input_dim=vocab_size,
      output_dim=hp.Int('embedding_dim', min_value=8, max_value=32, step=8) # задаем диапазон изменения параметра embedding_dim
    ))
    model.add(layers.GlobalAveragePooling1D())

    # Задаем количество слоев, их размерности и функции активации
    for i in range(hp.Int('num_layers', 1, 4)): # число слоев
        model.add(layers.Dense(units=hp.Int('units_' + str(i), 32, 256, 32), # размерность
                               activation=hp.Choice('activation_' + str(i), ['relu', 'tanh', 'sigmoid']))) # подбор функции активации
    model.add(layers.Dense(10, activation='sigmoid')) # Выходной слой

    # Определение оптимизатора и скомпилирование модели
    # learning_rate - выбирается из списка [1e-3, 1e-4, 1e-5]
    model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', [1e-3, 1e-4, 1e-5])), loss=keras.losses.BinaryCrossentropy(), metrics=['accuracy'])

    return model

In [ ]:
# Определение гиперпараметров для настройки
tuner_search = RandomSearch(build_model,
                            objective='val_accuracy',
                            max_trials=10,
                            directory='output')

# Запуск настройки гиперпараметров
tuner_search.search(train_data, y_train, epochs=5, validation_split=0.2)

# Получение наилучшей модели
best_model = tuner_search.get_best_models(num_models=1)[0]

# Оценка наилучшей модели на тестовых данных
test_loss, test_accuracy = best_model.evaluate(test_data, y_test)
print('Test accuracy:', test_accuracy)

In [ ]:
best_model.summary()